In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, sum, avg, max, min, stddev, countDistinct, lit, log1p, to_date, hour, dayofweek, min as spark_min, max as spark_max
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
import os
import gdown

In [42]:
spark = SparkSession.builder.appName("Fraud Detection Features").getOrCreate()

raw_csv_path = "../data/raw/Final Transaction.csv"
raw_parquet_path = "../data/raw/transactions.parquet"
if not os.path.exists(raw_csv_path):
    url = "https://drive.google.com/file/d/1qJtOG4ReSYyaHn0xbitvXqESQS94OkqP/view?usp=drive_link"
    gdown.download(url, raw_csv_path, quiet=True, fuzzy=True)

if not os.path.exists(raw_parquet_path):
    df_raw = spark.read.csv(raw_csv_path, header=True, inferSchema=True)
    df_raw.write.mode("overwrite").parquet(raw_parquet_path)
    
df_features = spark.read.parquet(raw_parquet_path, quiet=True)
df_features.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- TRANSACTION_ID: integer (nullable = true)
 |-- TX_DATETIME: timestamp (nullable = true)
 |-- CUSTOMER_ID: integer (nullable = true)
 |-- TERMINAL_ID: integer (nullable = true)
 |-- TX_AMOUNT: double (nullable = true)
 |-- TX_TIME_SECONDS: integer (nullable = true)
 |-- TX_TIME_DAYS: integer (nullable = true)
 |-- TX_FRAUD: integer (nullable = true)
 |-- TX_FRAUD_SCENARIO: integer (nullable = true)



In [43]:
remove_cols = ['_c0', 'TRANSACTION_ID', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_FRAUD_SCENARIO']
target_col = ["TX_FRAUD"]

df_features = df_features.drop(*remove_cols)
df_features.printSchema()
df_features.show(5)
df_features.describe().show()
df_features.groupBy("TX_FRAUD").count().show()

root
 |-- TX_DATETIME: timestamp (nullable = true)
 |-- TX_AMOUNT: double (nullable = true)
 |-- TX_TIME_SECONDS: integer (nullable = true)
 |-- TX_TIME_DAYS: integer (nullable = true)
 |-- TX_FRAUD: integer (nullable = true)

+-------------------+---------+---------------+------------+--------+
|        TX_DATETIME|TX_AMOUNT|TX_TIME_SECONDS|TX_TIME_DAYS|TX_FRAUD|
+-------------------+---------+---------------+------------+--------+
|2023-01-01 00:00:31|   533.07|             31|           0|       0|
|2023-01-01 00:02:10|   808.56|            130|           0|       0|
|2023-01-01 00:07:56|  1442.94|            476|           0|       1|
|2023-01-01 00:09:29|   620.65|            569|           0|       0|
|2023-01-01 00:10:34|   490.66|            634|           0|       0|
+-------------------+---------+---------------+------------+--------+
only showing top 5 rows
+-------+------------------+-----------------+------------------+-------------------+
|summary|         TX_AMOUNT|  TX_

In [44]:
df_features = df_features.withColumn("TX_DATE", to_date("TX_DATETIME")) \
            .withColumn("TX_HOUR", hour("TX_DATETIME")) \
            .withColumn("TX_DAYOFWEEK", dayofweek("TX_DATETIME"))

non_features = ["TX_DATETIME"]

df_features = df_features.drop(*non_features)
df_features.printSchema()
df_features.show(5)

root
 |-- TX_AMOUNT: double (nullable = true)
 |-- TX_TIME_SECONDS: integer (nullable = true)
 |-- TX_TIME_DAYS: integer (nullable = true)
 |-- TX_FRAUD: integer (nullable = true)
 |-- TX_DATE: date (nullable = true)
 |-- TX_HOUR: integer (nullable = true)
 |-- TX_DAYOFWEEK: integer (nullable = true)

+---------+---------------+------------+--------+----------+-------+------------+
|TX_AMOUNT|TX_TIME_SECONDS|TX_TIME_DAYS|TX_FRAUD|   TX_DATE|TX_HOUR|TX_DAYOFWEEK|
+---------+---------------+------------+--------+----------+-------+------------+
|   533.07|             31|           0|       0|2023-01-01|      0|           1|
|   808.56|            130|           0|       0|2023-01-01|      0|           1|
|  1442.94|            476|           0|       1|2023-01-01|      0|           1|
|   620.65|            569|           0|       0|2023-01-01|      0|           1|
|   490.66|            634|           0|       0|2023-01-01|      0|           1|
+---------+---------------+------------+-

In [45]:
df_features = (
    df_features
    .withColumn("LOG_TX_AMOUNT", log1p("TX_AMOUNT"))
    .withColumn("IS_NIGHT", when(col("TX_HOUR").between(0, 6), 1).otherwise(0))
    .withColumn("IS_WEEKEND", when(col("TX_DAYOFWEEK").isin([1, 7]), 1).otherwise(0))
)


In [46]:
feature_cols = [
    "TX_AMOUNT",
    "LOG_TX_AMOUNT",
    "TX_TIME_SECONDS",
    "TX_TIME_DAYS",
    "TX_HOUR",
    "IS_NIGHT",
    "IS_WEEKEND",
    "TX_FRAUD"
]

df_model = df_features.select(feature_cols + ["TX_DATE"])
df_model.printSchema()
df_model.show(5)


root
 |-- TX_AMOUNT: double (nullable = true)
 |-- LOG_TX_AMOUNT: double (nullable = true)
 |-- TX_TIME_SECONDS: integer (nullable = true)
 |-- TX_TIME_DAYS: integer (nullable = true)
 |-- TX_HOUR: integer (nullable = true)
 |-- IS_NIGHT: integer (nullable = false)
 |-- IS_WEEKEND: integer (nullable = false)
 |-- TX_FRAUD: integer (nullable = true)
 |-- TX_DATE: date (nullable = true)

+---------+-----------------+---------------+------------+-------+--------+----------+--------+----------+
|TX_AMOUNT|    LOG_TX_AMOUNT|TX_TIME_SECONDS|TX_TIME_DAYS|TX_HOUR|IS_NIGHT|IS_WEEKEND|TX_FRAUD|   TX_DATE|
+---------+-----------------+---------------+------------+-------+--------+----------+--------+----------+
|   533.07|6.280526916511479|             31|           0|      0|       1|         1|       0|2023-01-01|
|   808.56|6.696490890198005|            130|           0|      0|       1|         1|       0|2023-01-01|
|  1442.94|7.275130767342957|            476|           0|      0|       1| 

In [47]:
df_model.write.mode("overwrite").parquet("../data/features/fraud_features_v2.parquet")

26/01/01 23:08:26 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
26/01/01 23:08:26 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
26/01/01 23:08:26 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
26/01/01 23:08:26 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
26/01/01 23:08:26 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [51]:
df_model.select(max("TX_DATE").alias("MAX_TX_DATE"), min("TX_DATE").alias("MIN_TX_DATE")).show()

+-----------+-----------+
|MAX_TX_DATE|MIN_TX_DATE|
+-----------+-----------+
| 2023-07-02| 2023-01-01|
+-----------+-----------+

